In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def connect_linkedin(browser, url, email, subject, message, file, min_delay):
    browser.get(url)
    time.sleep(random.uniform(min_delay, min_delay + 3))

    #Customize message
    try:
        panel_left = browser.find_element_by_xpath('//div[contains(@class, "pv-text-details__left-panel")]').text
        name = panel_left.split('\n')[0]
        for i in [' (He/Him)', ' (She/Her)']:
            name = name.replace(i, '')

    except:
        name = ''
        pass

    subject = subject.replace('@subject', name)
    message = message.replace('@name', name)

    #Connect stay in More
    element_list = browser.find_elements_by_xpath('//button[contains(@aria-label, "More actions")]')
    for elementID in element_list:
        try:
            elementID.click()
            break
        except:
            pass

    #Clicks Connect button
    element_list = browser.find_elements_by_xpath('//*[text()="Connect"]')
    for elementID in element_list:
        try:
            elementID.click()
            break
        except:
            pass

    try:
        browser.find_element_by_xpath('//button[contains(@aria-label, "Connect")]').click()
    except:
        pass

    time.sleep(2)

    try:
        browser.find_element_by_xpath('//button[contains(@aria-label, "Add a note")]').click()
    except:
        pass

    actions = ActionChains(browser)
    for part in message.split('\n'):
        actions.send_keys(part)
        actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT)
        time.sleep(2)

    actions.perform()
    time.sleep(2)

    # Add email to send
    try:
        browser.find_element_by_name('email').send_keys(email)
    except:
        pass

    send_button = browser.find_element_by_class_name('ml1')

    clickable = False
    time_wait = time.time()
    while not clickable and time.time() < time_wait + implicit_time_default:
        try:
            cursor = send_button.value_of_css_property('cursor')
            if cursor != 'not-allowed':
                clickable = True
            break
        except:
            continue

    if clickable:
        send_button.click()

    time.sleep(2)
    ActionChains(browser).send_keys(Keys.ESCAPE).perform()

    return name


def connect_linkedin_via_email(browser, email_list, min_delay):
    browser.get('https://www.linkedin.com/mynetwork/import-contacts/iwe/')
    time.sleep(random.uniform(min_delay, min_delay + 3))
    browser.find_element_by_xpath('//textarea[contains(@name, "message")]').click()

    actions = ActionChains(browser)
    actions.send_keys(email_list)
    actions.perform()
    time.sleep(2)

    browser.find_element_by_xpath('//button[contains(@id, "send-iwe")]').click()


def check_connection(browser, username, password, target_profile_url):
    api = Linkedin(username, password)
    target_profile = api.get_profile(target_profile_url)
    connections = api.get_connections(username)

    if target_profile.id in [c['id'] for c in connections]:
        return True
    else:
        return False

In [ ]:
def send_linkedin(browser, url, email, subject, message, file, min_delay):
    browser.get(url)
    time.sleep(random.uniform(min_delay, min_delay + 3))

    #Check if connected
    try:
        browser.find_element_by_xpath('//*[text()="Remove Connection"]')
    except:
        raise Exception('Contact is not connected.')

    #Customize message
    try:
        panel_left = browser.find_element_by_xpath('//div[contains(@class, "pv-text-details__left-panel")]').text
        name = panel_left.split('\n')[0]
    except:
        name = ''
        pass

    subject = subject.replace('@subject', name)
    message = message.replace('@name', name)
	# thêm code ở chổ này
    elements=browser.find_elements_by_xpath('//button[contains(@aria-label,"Message")]')
    button_message= elements[1]
    button_message.click()
    try:
        elementID = browser.find_element_by_xpath('//a[contains(@class, "app-aware-link") and contains(@class,"artdeco-button") and contains(@class,"artdeco-button--premium")]')

        if elementID.get_property('href') != 'https://www.linkedin.com/premium/my-premium/':
            browser.get(elementID.get_property('href'))
            time.sleep(2)

            #Attach images
            try:
                elementID = browser.find_element_by_xpath(
                    '//input[@accept="image/*,.ai,.psd,.pdf,.doc,.docx,.csv,.zip,.rar,.ppt,.pptx,.pps,.ppsx,'+
                    '.odt,.rtf,.xls,.xlsx,.txt,.pub,.html,.7z,.eml"]'
                )
                for f in file:
                    elementID.send_keys(util.path_media + f)
                    time.sleep(2)
            except:
                pass

            try:
                browser.find_element_by_xpath('//input[@name="subject"]').send_keys(subject)
            except:
                pass

            browser.find_element_by_xpath('//div[@role="textbox" and @aria-multiline="true"]').click()
            # msg-form__contenteditable t-14 t-black--light t-normal flex-grow-1 full-height notranslate


            actions = ActionChains(browser)
            for part in message.split('\n'):
                actions.send_keys(part)
                actions.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER)
                time.sleep(2)

            actions.perform()
            time.sleep(2)

            send_button = browser.find_element_by_class_name('msg-form__send-button')

            clickable = False
            time_wait = time.time()
            while not clickable and time.time() < time_wait + implicit_time_default:
                try:
                    cursor = send_button.value_of_css_property('cursor')
                    if cursor != 'not-allowed':
                        clickable = True
                    break
                except:
                    continue

            if clickable:
                send_button.click()

    except:
        pass


    #If Follow is required
#        browser.find_element_by_class_name('pv-s-profile-actions--follow').click()

    # Connect
    # checkSentButton = browser.find_element_by_class_name('ml1')
    # if 'artdeco-button--disabled' in checkSentButton.get_attribute('class').split():
    #     browser.find_element_by_class_name('artdeco-modal__dismiss').click()
    #     browser.find_element_by_class_name('pv-s-profile-actions--message').click()
    #     if True == check_exists_class('msg-form__subject'):
    #         browser.find_element_by_class_name('msg-form__subject').send_keys(subject)
    #     if True == check_exists_class('msg-form__contenteditable'):
    #         browser.find_element_by_class_name('msg-form__contenteditable').send_keys(message)
    #     if True == check_exists_class('msg-form__send-button'):
    #         browser.find_element_by_class_name('msg-form__send-button').click()
    # else:
    #     checkSentButton.click()

    time.sleep(2)
    ActionChains(browser).send_keys(Keys.ESCAPE).perform()

    return name

In [ ]:
# import tweepy
# import os

# # Thông tin đăng nhập Twitter từ GitHub Secrets
# consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
# consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
# access_token = os.getenv('TWITTER_ACCESS_TOKEN')
# access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

# # Xác thực với Twitter
# auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
# api = tweepy.API(auth)

# # Đăng bài lên Twitter
# tweet = "Hello, world! This is an automated tweet."
# api.update_status(status=tweet)
